In [ ]:
%matplotlib inline

import datetime

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

import utide

print(utide.__version__)

In [ ]:
# prompt: !pip install utide

import os

# Check if UTide directory exists, if not, clone it
if not os.path.exists('UTide'):
    !git clone https://github.com/wesleybowman/UTide.git

# Add UTide to the Python path
import sys
sys.path.insert(0, 'UTide')

import utide

utide.__version__

In [ ]:
!pip install utide

In [ ]:
with open("./notebooks/can1998.dtf") as f:
    lines = f.readlines()

print("".join(lines[:5]))

In [ ]:
names = ["seconds", "year", "month", "day", "hour", "elev", "flag"]

obs = pd.read_csv(
    "./notebooks/can1998.dtf",
    names=names,
    skipinitialspace=True,
    delim_whitespace=True,
    na_values="9.990",
)


date_cols = ["year", "month", "day", "hour"]
index = pd.to_datetime(obs[date_cols])
obs = obs.drop(date_cols, axis=1)
obs.index = index

obs.head(5)

In [ ]:
bad = obs["flag"] == 2
corrected = obs["flag"] == 1

obs.loc[bad, "elev"] = np.nan
obs["anomaly"] = obs["elev"] - obs["elev"].mean()
obs["anomaly"] = obs["anomaly"].interpolate()
print(f"{bad.sum()} points were flagged 'bad' and interpolated")
print(f"{corrected.sum()} points were flagged 'corrected' and left unchanged")

In [ ]:
coef = utide.solve(
    obs.index,
    obs["anomaly"],
    lat=-25,
    method="ols",
    conf_int="MC",
    verbose=False,
)

In [ ]:
print(coef.keys())

In [ ]:
tide = utide.reconstruct(obs.index, coef, verbose=False)

In [ ]:
print(tide.keys())

In [ ]:
t = obs.index.to_pydatetime()

fig, (ax0, ax1, ax2) = plt.subplots(figsize=(17, 5), nrows=3, sharey=True, sharex=True)

ax0.plot(t, obs.anomaly, label="Observations", color="C0")
ax1.plot(t, tide.h, label="Prediction", color="C1")
ax2.plot(t, obs.anomaly - tide.h, label="Residual", color="C2")
fig.legend(ncol=3, loc="upper center");

In [ ]:
last_month = obs.loc['1998-12-01':'1999-01-01']
# 确保 last_month.index 与 obs.index 对齐，并找到索引位置
index_positions = obs.index.get_indexer(last_month.index)

# 使用这些索引位置从 tide.h 中提取对应的预测数据
pred_last_month = tide.h[index_positions]
t_last = last_month.index.to_pydatetime()
#pred_last_month = tide.h[last_month.index]
fig, (ax0, ax1, ax2) = plt.subplots(figsize=(17, 5), nrows=3, sharey=True, sharex=True)
plt.plot(t_last, last_month.anomaly, label="Observations", color="C0")
plt.plot(t_last, pred_last_month, label="Prediction", color="C1")
plt.legend()
plt.show()

In [ ]:
# 确保 last_month.index 与 obs.index 对齐，并找到索引位置
index_positions = obs.index.get_indexer(last_month.index)

# 使用这些索引位置从 tide.h 中提取对应的预测数据
pred_last_month = tide.h[index_positions]

# 计算最后一个月的残差
residual_last_month = last_month.anomaly - pred_last_month

# 绘制图像
t_last = last_month.index.to_pydatetime()
fig, (ax0, ax1, ax2) = plt.subplots(figsize=(17, 5), nrows=3, sharey=True, sharex=True)

ax0.plot(t_last, last_month.anomaly, label="Observations", color="C0")
ax1.plot(t_last, pred_last_month, label="Prediction", color="C1")
ax2.plot(t_last, residual_last_month, label="Residual", color="C2")

fig.legend(ncol=3, loc="upper center")
plt.show()

In [ ]:
import numpy as np

# 确保 last_month.index 与 obs.index 对齐，并找到索引位置
index_positions = obs.index.get_indexer(last_month.index)

# 使用这些索引位置从 tide.h 中提取对应的预测数据
pred_last_month = tide.h[index_positions]

# 计算最后一个月的残差
residual_last_month = last_month.anomaly - pred_last_month

# 计算残差的平方（即误差平方）
mse_last_month = residual_last_month ** 2

# 计算整个时间段的平均 MSE
avg_mse = np.mean(mse_last_month)
print(f"Average MSE for last month: {avg_mse}")

# 绘制图像
t_last = last_month.index.to_pydatetime()
fig, (ax0, ax1, ax2, ax3) = plt.subplots(figsize=(17, 8), nrows=4, sharey=False, sharex=True)

ax0.plot(t_last, last_month.anomaly, label="Observations", color="C0")
ax1.plot(t_last, pred_last_month, label="Prediction", color="C1")
ax2.plot(t_last, residual_last_month, label="Residual", color="C2")
ax3.plot(t_last, mse_last_month, label="MSE", color="C3")

fig.legend(ncol=4, loc="upper center")
plt.show()
